In [11]:
from transformers import pipeline
import logging

In [12]:
# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LumiSentimentModel")

In [13]:
# Load pre-trained pipelines
logger.info("Loading sentiment and emotion analysis models...")
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

Device set to use cpu
Device set to use cpu


In [14]:
# Keywords and phrases indicating harmful intent
HARMFUL_KEYWORDS = [
    "suicide", "kill myself", "want to die", "no reason to live",
    "end it all", "can't go on", "worthless", "hopeless",
    "disappear forever", "I hate myself", "self-harm"
]

In [15]:
def analyze_entry(text):
    """Analyze sentiment and emotion."""
    sentiment = sentiment_pipeline(text)[0]
    emotions = emotion_pipeline(text)[0]
    emotion_sorted = sorted(emotions, key=lambda x: x['score'], reverse=True)

    top_emotion = emotion_sorted[0]['label']
    confidence = emotion_sorted[0]['score']

    return {
        "sentiment": sentiment['label'],
        "sentiment_score": sentiment['score'],
        "top_emotion": top_emotion,
        "emotion_confidence": confidence,
        "emotions_full": emotions
    }

In [16]:
def detect_harmful_language(text):
    """Detect harmful/self-harm language."""
    text_lower = text.lower()
    for phrase in HARMFUL_KEYWORDS:
        if phrase in text_lower:
            return True
    return False

In [17]:
def process_journal_entry(entry_text):
    """Process journal entry and analyze emotional state."""
    logger.info("Processing journal entry...")
    analysis = analyze_entry(entry_text)
    is_harmful = detect_harmful_language(entry_text)

    result = {
        "analysis": analysis,
        "harmful_language_detected": is_harmful
    }

    if is_harmful:
        logger.warning("Harmful language detected! Emergency response may be needed.")
        # Here you'd trigger a real alert to your backend system or human reviewer.

    return result

In [18]:
# Example usage
if __name__ == "__main__":
    sample_journals = [
        "I had such an amazing day at the park today! I laughed so much and felt truly at peace.",
        "Everyone got a promotion except me. I'm trying to be happy for them, but it's hard not to feel jealous.",
        "I don’t even know what’s wrong with me. I just feel so empty lately.",
        "Life is beautiful and I’m so grateful for my friends and family.",
        "I can't do this anymore. It all feels like too much to handle.",
        "My heart is pounding—I’m so excited for tomorrow’s presentation!"
    ]

In [19]:
for entry in sample_journals:
  print(f"\nJournal Entry: {entry}")
  output = process_journal_entry(entry)
  print("Analysis Result:", output)


Journal Entry: I had such an amazing day at the park today! I laughed so much and felt truly at peace.
Analysis Result: {'analysis': {'sentiment': 'POSITIVE', 'sentiment_score': 0.9998513460159302, 'top_emotion': 'joy', 'emotion_confidence': 0.9861605167388916, 'emotions_full': [{'label': 'anger', 'score': 0.00095271848840639}, {'label': 'disgust', 'score': 0.0010707201436161995}, {'label': 'fear', 'score': 0.0002452797198202461}, {'label': 'joy', 'score': 0.9861605167388916}, {'label': 'neutral', 'score': 0.005490459036082029}, {'label': 'sadness', 'score': 0.0041456823237240314}, {'label': 'surprise', 'score': 0.001934508327394724}]}, 'harmful_language_detected': False}

Journal Entry: Everyone got a promotion except me. I'm trying to be happy for them, but it's hard not to feel jealous.
Analysis Result: {'analysis': {'sentiment': 'POSITIVE', 'sentiment_score': 0.9978803396224976, 'top_emotion': 'anger', 'emotion_confidence': 0.5457968711853027, 'emotions_full': [{'label': 'anger', 

In [21]:
# Save sentiment model and emotion model
sentiment_pipeline.model.save_pretrained("/content/sentiment_model")
sentiment_pipeline.tokenizer.save_pretrained("/content/sentiment_model")

emotion_pipeline.model.save_pretrained("/content/emotion_model")
emotion_pipeline.tokenizer.save_pretrained("/content/emotion_model")


('/content/emotion_model/tokenizer_config.json',
 '/content/emotion_model/special_tokens_map.json',
 '/content/emotion_model/vocab.json',
 '/content/emotion_model/merges.txt',
 '/content/emotion_model/added_tokens.json',
 '/content/emotion_model/tokenizer.json')